In [19]:
from transformers import BertConfig, BertModel, AutoTokenizer, TrainingArguments, AutoModelForSequenceClassification, Trainer, DataCollatorWithPadding
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import torch
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict, ClassLabel
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
import seaborn as sns

import matplotlib.pyplot as plt
from collections import Counter


In [54]:
torch.cuda.is_available()

True

In [55]:
tokenizer = AutoTokenizer.from_pretrained("TurkuNLP/bert-base-finnish-cased-v1")

In [56]:
# Loading dataset
dataSetPath = "yle_data/2011-2018-SubjectsInPaper-1.csv"
df = pd.read_csv(dataSetPath, sep="␞", nrows=200000)


print(df['text'].isna().sum(), "invalid data")
df = df.dropna(subset=['text'])
print(len(df), "valid data")

print(df.columns)
df = df.drop(['id', 'url', 'headline', 'datePublished'], axis=1) 


def multiLabelToSingular(labelArray):
    cleaned_text = labelArray.replace("'", "").replace(" ", "").replace("[", "").replace("]", "")
    return cleaned_text.split(",")[0]


# print(df.dtypes)
df['subjects'] = df['subjects'].apply(multiLabelToSingular)

print(df['subjects'].nunique(), "categories")
print(df['subjects'].unique().tolist())

df.rename({'subjects': 'labels'}, axis="columns", inplace=True)



C:\Users\joakim\AppData\Local\Temp\ipykernel_20584\3043923669.py:3: ParserWarning: Falling back to the 'python' engine because the separator encoded in utf-8 is > 1 char long, and the 'c' engine does not support such separators; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(dataSetPath, sep="␞", nrows=200000)


612 invalid data
199388 valid data
Index(['id', 'url', 'headline', 'text', 'subjects', 'datePublished'], dtype='object')
10 categories
['urheilu', 'kulttuuri', 'talous', 'luonto', 'Onnettomuudet', 'politiikka', 'Rikokset', 'Liikenne_ja_kuljetus', 'Koulutus_ja_kasvatus', 'Terveys']


In [ ]:
# paperin luokat:

# urheilu "18-220090",
# talous "18-204933",
# politiikka "18-220306",
# kulttuuri "18-208149",
# luonto "18-215452",
# 18-218480 - Onnettomuudet
# 18-209306 - Terveys
# 18-217206 - Rikokset
# 18-91232 - Liikenne ja kuljetus
# 18-35286 - Koulutus ja kasvatus

label_mapping = {
    "Koulutus_ja_kasvatus": 0,
    "Liikenne_ja_kuljetus": 1,
    "Onnettomuudet": 2,
    "Rikokset": 3,
    "Terveys": 4,
    "kulttuuri": 5,
    "luonto": 6,
    "politiikka": 7,
    "talous": 8,
    "urheilu": 9
}

to_labels = {y: x for x, y in label_mapping.items()}

print(df['labels'].value_counts().sort_index(ascending=True))
df["labels"] = df["labels"].map(label_mapping)
print(df['labels'].value_counts().sort_index(ascending=True))


Koulutus_ja_kasvatus    10753
Liikenne_ja_kuljetus    13554
Onnettomuudet           10717
Rikokset                 8166
Terveys                  8600
kulttuuri               13794
luonto                  10994
politiikka              12961
talous                  24427
urheilu                 85422
Name: labels, dtype: int64
0    10753
1    13554
2    10717
3     8166
4     8600
5    13794
6    10994
7    12961
8    24427
9    85422
Name: labels, dtype: int64


In [58]:
n_sample = 250
df_balanced = df.groupby('labels').head(n_sample)
df_balanced = df.groupby('labels').apply(lambda x: x.sample(n=n_sample))

print(len(df_balanced), "documents")
print(df_balanced['labels'].value_counts().sort_index(ascending=True))


2500 documents
0    250
1    250
2    250
3    250
4    250
5    250
6    250
7    250
8    250
9    250
Name: labels, dtype: int64


In [59]:
dataset = Dataset.from_pandas(df_balanced, preserve_index=False)
dataset

Dataset({
    features: ['text', 'labels'],
    num_rows: 2500
})

In [60]:

train_test_split = dataset.train_test_split(test_size=0.2)

train_valid_split = train_test_split['train'].train_test_split(test_size=0.1)

datasets = DatasetDict({
    'train': train_valid_split['train'],
    'validation': train_valid_split['test'],
    'test': train_test_split['test']
})
print(datasets)

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 1800
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 200
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 500
    })
})


In [61]:
unique_values = set(datasets["train"]["labels"])
print(unique_values)


{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


In [62]:
def tokenize_function(example):
    tokenized = tokenizer(example["text"], truncation=True)
    return tokenized

In [63]:

tokenized_datasets = datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

tokenized_datasets



Map:   0%|          | 0/1800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1800
    })
    validation: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 200
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 500
    })
})

In [64]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {"accuracy": accuracy, "f1": f1}

training_args = TrainingArguments(
    output_dir="./logs",                    
    evaluation_strategy="steps",        
    eval_steps=50,                      
    logging_dir="./logs",                
    logging_steps=50,                   
    save_steps=500,                      
    per_device_train_batch_size=8,       
    per_device_eval_batch_size=16,       
    num_train_epochs=5,                  
    save_total_limit=2,                  
    load_best_model_at_end=True,         
    metric_for_best_model="f1",          
    greater_is_better=True,              
    report_to="tensorboard",            
)





In [65]:
model = AutoModelForSequenceClassification.from_pretrained("TurkuNLP/bert-base-finnish-cased-v1", num_labels=11, device_map = 'cuda')


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at TurkuNLP/bert-base-finnish-cased-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\joakim\anaconda3\lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(


In [66]:
trainer.train()

  0%|          | 0/1125 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 1.7289, 'learning_rate': 4.7777777777777784e-05, 'epoch': 0.22}


  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 1.1251718997955322, 'eval_accuracy': 0.675, 'eval_f1': 0.6437577030812325, 'eval_runtime': 24.603, 'eval_samples_per_second': 8.129, 'eval_steps_per_second': 0.528, 'epoch': 0.22}
{'loss': 0.9041, 'learning_rate': 4.555555555555556e-05, 'epoch': 0.44}


...

  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 0.722608208656311, 'eval_accuracy': 0.87, 'eval_f1': 0.8699085091929085, 'eval_runtime': 31.4343, 'eval_samples_per_second': 6.362, 'eval_steps_per_second': 0.414, 'epoch': 4.89}


c:\Users\joakim\anaconda3\lib\site-packages\transformers\trainer.py:2189: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(best_model_path, map_location

{'train_runtime': 3435.6931, 'train_samples_per_second': 2.62, 'train_steps_per_second': 0.327, 'train_loss': 0.33906754088401797, 'epoch': 5.0}


TrainOutput(global_step=1125, training_loss=0.33906754088401797, metrics={'train_runtime': 3435.6931, 'train_samples_per_second': 2.62, 'train_steps_per_second': 0.327, 'train_loss': 0.33906754088401797, 'epoch': 5.0})

In [67]:
predictions = trainer.predict(test_dataset=tokenized_datasets["test"])
print(predictions.predictions.shape, predictions.label_ids.shape)


  0%|          | 0/32 [00:00<?, ?it/s]

(500, 11) (500,)


In [68]:

preds = np.argmax(predictions.predictions, axis=-1)
print(preds)
print(predictions.label_ids)

accuracy = accuracy_score(predictions.label_ids, preds)
f1 = f1_score(predictions.label_ids, preds, average='weighted')
print("accuracy=" , accuracy, "f1=" , f1)



[1 3 7 4 7 5 3 3 5 6 1 5 2 2 8 2 5 1 2 5 2 9 4 2 4 4 0 7 8 7 3 8 2 8 0 4 9
 2 5 7 2 8 6 2 8 0 3 8 9 2 4 0 6 0 9 9 9 1 2 0 5 2 8 0 4 0 1 9 1 0 2 3 5 4
 7 2 7 0 4 0 9 3 9 5 6 0 9 3 8 6 4 6 4 8 0 3 4 2 2 7 6 0 8 2 5 6 3 4 0 9 3
 0 5 5 9 9 2 5 2 3 4 4 6 1 8 2 2 3 5 4 2 3 1 0 3 3 8 5 2 4 8 2 2 1 8 0 2 0
 6 7 6 6 5 5 1 3 7 0 4 5 7 6 0 7 1 7 3 2 5 4 3 7 4 6 7 1 6 9 5 4 0 6 7 4 3
 0 8 1 8 6 2 0 9 7 9 7 9 9 1 2 8 6 8 6 5 1 7 7 1 4 6 4 5 4 2 2 3 3 2 3 5 3
 3 8 5 4 7 3 0 7 1 0 2 2 5 7 3 2 6 9 9 2 0 1 7 5 3 0 3 7 3 2 7 0 3 2 8 0 8
 6 9 2 9 7 2 5 5 9 3 7 6 0 6 1 0 1 3 3 4 6 4 3 2 0 1 9 0 4 0 8 1 6 4 7 8 3
 6 2 3 1 9 9 6 9 8 5 8 6 0 4 9 6 3 1 2 3 2 4 4 9 2 1 3 7 3 1 2 2 9 6 9 7 1
 7 8 4 4 3 5 5 2 8 0 5 0 6 0 7 3 1 9 0 0 5 3 9 3 5 3 3 4 4 6 3 5 2 1 2 5 7
 5 1 8 3 0 6 0 2 3 2 9 7 4 8 7 6 6 4 2 8 3 2 0 6 3 0 3 0 7 7 7 5 3 2 3 0 0
 1 7 9 7 4 0 9 1 5 4 5 0 9 5 5 6 7 9 9 5 5 0 1 1 6 2 6 6 2 9 1 9 4 7 9 2 0
 9 8 9 8 5 5 5 6 0 8 4 0 4 8 3 2 9 5 1 2 6 6 0 5 5 1 8 1 3 8 5 3 1 4 9 3 0
 4 3 2 8 6 1 5 2 5 4 2 4 

In [69]:
trainer.evaluate(tokenized_datasets["test"])



  0%|          | 0/32 [00:00<?, ?it/s]

{'eval_loss': 0.7723776698112488,
 'eval_accuracy': 0.86,
 'eval_f1': 0.8583619487885783,
 'eval_runtime': 82.9164,
 'eval_samples_per_second': 6.03,
 'eval_steps_per_second': 0.386,
 'epoch': 5.0}

In [70]:
# Loading dataset
test_corpus_data = "test_corpus_data.csv" # 63 dokumenttia
df_corpus = pd.read_csv(test_corpus_data, sep="|")

df_corpus.head(3)


,text_modernized,text_better_ocr,labels
0,Kuulumiset ulkomailta ovat olleet hyvin vähäis...,Kuulumiset ulkomailta owat olleet hywin wähäi-...,Rikokset
1,"Amerikasta kerrotaan, että New Yorkiin, Amerik...","Amerikasta kerrotaan, että NewYork'iin Amerika...",Liikenne_ja_kuljetus
2,Kurmijärveltä kirjoitetaan meille joulukuun lo...,Kurmijärweltä kirjoitetaan meille lopulla wii-...,Rikokset


In [71]:
df_corpus['labels'] = df_corpus.labels.map(label_mapping)
print(df_corpus.labels)

df_corpus_modernized = df_corpus.copy()

df_corpus.rename({'text_better_ocr': 'text'}, axis="columns", inplace=True) # parempi ocr-teksti on kopioitu kansalliskirjaston sivuilta. alkuperäisen datasetin ocr-data on heikkolaatuista 
df_corpus_modernized.rename({'text_modernized': 'text'}, axis="columns", inplace=True)


0     3
1     1
2     3
3     7
4     7
     ..
58    7
59    7
60    7
61    6
62    8
Name: labels, Length: 63, dtype: int64


In [72]:
df_corpus = Dataset.from_pandas(df_corpus, preserve_index=False)
df_corpus_modernized = Dataset.from_pandas(df_corpus_modernized, preserve_index=False)



In [73]:
tokenized_corpus = df_corpus.map(tokenize_function, batched=True)
tokenized_corpus

tokenized_corpus_modern = df_corpus_modernized.map(tokenize_function, batched=True)
tokenized_corpus_modern


Map:   0%|          | 0/63 [00:00<?, ? examples/s]

Map:   0%|          | 0/63 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'text_better_ocr', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 63
})

In [74]:
predictions_corpus = trainer.predict(test_dataset=tokenized_corpus)
preds_corpus = np.argmax(predictions_corpus.predictions, axis=-1)
print(preds_corpus)
print(predictions_corpus.label_ids)

accuracy = accuracy_score(predictions_corpus.label_ids, preds_corpus)
f1 = f1_score(predictions_corpus.label_ids, preds_corpus, average='weighted')
print("accuracy=" , accuracy, "f1=" , f1)

  0%|          | 0/4 [00:00<?, ?it/s]

[0 8 6 7 7 7 7 7 8 1 7 5 8 5 7 8 5 0 5 8 8 8 1 8 5 5 0 5 7 7 7 5 8 8 0 0 8
 7 8 8 5 8 5 8 8 8 8 5 8 5 8 5 5 3 5 8 5 7 7 7 7 5 5]
[3 1 3 7 7 7 7 7 8 1 1 5 1 7 7 7 8 7 5 8 8 5 1 8 3 3 0 7 7 7 7 7 8 7 0 7 5
 7 8 8 5 8 6 5 8 8 8 7 8 5 1 5 5 1 6 1 8 7 7 7 7 6 8]
accuracy= 0.5714285714285714 f1= 0.5554207268492983


In [75]:
predictions_corpus = trainer.predict(test_dataset=tokenized_corpus_modern)
preds_corpus = np.argmax(predictions_corpus.predictions, axis=-1)
print(preds_corpus)
print(predictions_corpus.label_ids)

accuracy = accuracy_score(predictions_corpus.label_ids, preds_corpus)
f1 = f1_score(predictions_corpus.label_ids, preds_corpus, average='weighted')
print("accuracy=" , accuracy, "f1=" , f1)

  0%|          | 0/4 [00:00<?, ?it/s]

[0 8 6 8 7 7 7 7 8 1 5 5 8 8 7 8 8 0 5 8 8 8 1 3 3 3 0 5 7 7 7 5 8 8 0 0 8
 7 8 8 5 1 6 8 8 8 8 3 8 5 8 5 5 1 6 1 1 7 7 7 7 7 5]
[3 1 3 7 7 7 7 7 8 1 1 5 1 7 7 7 8 7 5 8 8 5 1 8 3 3 0 7 7 7 7 7 8 7 0 7 5
 7 8 8 5 8 6 5 8 8 8 7 8 5 1 5 5 1 6 1 8 7 7 7 7 6 8]
accuracy= 0.6349206349206349 f1= 0.6421278062882073


In [76]:
model_save_path = "./t_model"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)


('./t_model\\tokenizer_config.json',
 './t_model\\special_tokens_map.json',
 './t_model\\vocab.txt',
 './t_model\\added_tokens.json',
 './t_model\\tokenizer.json')